In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 1000, 'display.width', 1000)

# Find professionals

In [3]:
pro = pd.read_csv('../../data/professionals.csv')
tags = pd.read_csv('../../data/tags.csv')
tag_pro = pd.read_csv('../../data/tag_users.csv')

In [4]:
tag_merged = tags.merge(tag_pro, left_on='tags_tag_id', right_on='tag_users_tag_id')
tag_grouped = tag_merged.groupby('tag_users_user_id')[['tags_tag_name']].aggregate(lambda x: ' '.join(x))

In [5]:
pro_merged = pro.merge(tag_grouped, left_on ='professionals_id', right_index = True)

In [6]:
pro_merged.sample(100)

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined,tags_tag_name
2572,56c20bb047c442b5a7f45c078aa16e2f,"Corpus Christi, Texas",Mechanical or Industrial Engineering,Mechanical Engineer Graduate seeking full time...,2015-11-09 19:01:23 UTC+0000,mechanical-engineering industrial-engineering
13335,7f133aff61ef424fa0e3e9c0e5d5202d,"Los Angeles, California",Accounting,Assurance Associate at PwC,2017-09-29 16:04:40 UTC+0000,accounting
10693,efccd7998414405e912aee58160d38cd,"California, California",Computer Software,"Senior Manager, Web Marketing at Okta, Inc.",2017-05-01 20:42:13 UTC+0000,computer-software
18225,29f3b121549a42b7a2cc5f88400a21de,"Austin, Texas",Information Technology and Services,"Director Of Business Development at Okta, Inc.",2018-04-10 19:02:42 UTC+0000,engineering computer electrical-engineering bu...
13680,a793a64cf2ff4a478aa72276b11d68c3,Dallas/Fort Worth Area,Education Management,"Principal, Instructional Designer, and Studen...",2017-10-16 00:54:16 UTC+0000,education-management
23957,81d4638e8adf437287c97f6ffa008446,"Salt Lake City, Utah",Biotechnology,Biological Engineer | Programmer-Analyst | Lab...,2018-10-03 20:11:28 UTC+0000,biomedical-engineering biotechnology microbiol...
6580,6392e3dffe8e4ca49542661820f8705f,Panama,Information Technology and Services,Audit Advisor at Dell Financial Services,2016-05-18 14:17:27 UTC+0000,information-technology
5810,e777f1189db54acc8ccb78f0cdcbbf2b,"Houston, Texas",Mechanical or Industrial Engineering,Mechanical Engineer,2016-05-04 00:46:04 UTC+0000,mechanical-engineering
6102,409e6ea809964c3da784c3ad6636db01,"Ocala, Florida",Marketing and Advertising,Recruitment Assistant at R+L Global Logistics,2016-05-09 16:36:05 UTC+0000,marketing-and-advertising
23125,0e12dcba85eb460b95675fd2312cf08d,"Jacksonville, Florida",Retail,Executive Assistant to the President of AT&T F...,2018-09-13 13:42:33 UTC+0000,retail #admin#eventpalnning#meetings#calendar ...


In [7]:
d = pro_merged[pro_merged['professionals_id'] == '1db1485cc8c74b0291505e92237b98d0'].reset_index().to_dict()
d = {key if key != 'tags_tag_name' else 'professionals_subscribed_tags': [val[0]] for key, val in d.items() if key != 'index'}
d

{'professionals_id': ['1db1485cc8c74b0291505e92237b98d0'],
 'professionals_location': ['New York, New York'],
 'professionals_industry': ['Hospital and Health Care'],
 'professionals_headline': ['Physical Therapist'],
 'professionals_date_joined': ['2016-05-16 15:39:07 UTC+0000'],
 'professionals_subscribed_tags': ['healthcare physical-therapist']}

# Prepare sample

In [9]:
id_sample = set(pro['professionals_id'].sample(50))

In [10]:
pro_sample = pro[pro['professionals_id'].isin(id_sample)]

In [11]:
tag_sample = tag_merged[tag_merged['tag_users_user_id'].isin(id_sample)]

In [12]:
from utils import TextProcessor

In [13]:
tp = TextProcessor()

In [14]:
for feature in ['professionals_industry', 'professionals_headline']:
    pro_sample[feature] = pro_sample[feature].apply(tp.process)

C:\Users\nikitosoleil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
tag_sample['tags_tag_name'] = tag_sample['tags_tag_name'].apply(lambda x: tp.process(x, allow_stopwords=True))

C:\Users\nikitosoleil\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
pro_sample.to_csv('sample/pro.csv')
tag_sample.to_csv('sample/tag_users.csv')

# Find questions

In [17]:
que = pd.read_csv('../../data/questions.csv')
tags = pd.read_csv('../../data/tags.csv')
tag_que = pd.read_csv('../../data/tag_questions.csv')

In [18]:
tag_merged = tags.merge(tag_que, left_on='tags_tag_id', right_on='tag_questions_tag_id')
tag_grouped = tag_merged.groupby('tag_questions_question_id')[['tags_tag_name']].aggregate(lambda x: ' '.join(x))

In [ ]:
que_merged = que.merge(tag_grouped, left_on ='questions_id', right_index = True)

In [ ]:
que_merged

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,tags_tag_name
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26 UTC+0000,Teacher career question,What is a maths teacher? what is a ma...,college professor lecture
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25 UTC+0000,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...,military army
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38 UTC+0000,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....,working-abroad overseas
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32 UTC+0000,To become a specialist in business management...,i hear business management is a hard way to ge...,business networking
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54 UTC+0000,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...,college scholarships firstgeneration highschoo...
5,7c336403258f4da3a2e0955742c76462,d1e4587c0e784c62bc27eb8d16a07f38,2017-03-01 04:27:08 UTC+0000,How many years of coege do you need to be an e...,To be an engineer #united-states,united-states
6,be3c5edfdb07423e955e9b2d7f186bce,71b4554d4a824253aa28287372c55797,2017-09-01 04:59:38 UTC+0000,I want to become a doctor because of my great ...,I am a musician and want to pursue that in col...,medicine music
7,0f1d6a4f276c4a05878dd48e03e52289,585ac233015447cc9e9a217044e515e1,2016-05-19 22:16:25 UTC+0000,what kind of college could i go to for a soc...,I like soccer because i been playing sense i w...,college soccer building
8,d4999cdc470049a1a3382c4b5f14a7aa,654e1d6fd5b947249c0e70658d01b2ac,2017-08-31 19:20:47 UTC+0000,What are the college classes like for and grap...,I'm asking because I was thinking about choosi...,graphic-design graphics
9,e214acfbe6644d65b889a3268828db9d,16908136951a48ed942738822cedd5c2,2012-09-09 05:33:25 UTC+0000,what does it take to be an anesthesiologist?,I am a sophomore who is interested in learning...,doctor healthcare experience anesthesiologist


In [ ]:
d = que_merged[que_merged['questions_id'] == 'd6ee6363b5f7486eb362bc615630fda8'].reset_index().to_dict()
d = {key if key != 'tags_tag_name' else 'questions_tags': [val[0]] for key, val in d.items() if key != 'index'}
d

{'questions_id': ['d6ee6363b5f7486eb362bc615630fda8'],
 'questions_author_id': ['02946e467bab4fd794e42f9670cb4279'],
 'questions_date_added': ['2017-07-29 13:30:50 UTC+0000'],
 'questions_title': ['I want to study law but not sure what subjects need to be taken,so need some advice on that.'],
 'questions_body': ['#law-practice #lawyer #career-details'],
 'questions_tags': ['lawyer law-practice career-details']}

# Students sample

In [ ]:
pd.read_csv('../../data/students.csv')

,students_id,students_location,students_date_joined
0,12a89e96755a4dba83ff03e03043d9c0,NaN,2011-12-16 14:19:24 UTC+0000
1,e37a5990fe354c60be5e87376b08d5e3,NaN,2011-12-27 03:02:44 UTC+0000
2,12b402cceeda43dcb6e12ef9f2d221ea,NaN,2012-01-01 05:00:00 UTC+0000
3,a0f431fc79794edcb104f68ce55ab897,NaN,2012-01-01 05:00:00 UTC+0000
4,23aea4702d804bd88d1e9fb28074a1b4,NaN,2012-01-01 05:00:00 UTC+0000
5,18a8f9363cd24a37b690e1b205146b14,NaN,2012-01-01 05:00:00 UTC+0000
6,d21c67279ada49d1bcf66ad620c00911,NaN,2012-01-01 05:00:00 UTC+0000
7,433c0f5f90344453ba50ec8aee5fdfa4,NaN,2012-01-01 05:00:00 UTC+0000
8,f0a7d23c3d374f8d9a1a5eb9b96eab0f,NaN,2012-01-01 05:00:00 UTC+0000
9,26269f6765b74347b28155aea1b57393,NaN,2012-01-01 05:00:00 UTC+0000
